Project 1 - Terrence Cummings
- Crime, Schools, and Home Values in Minneapolis

Step 1 - Housing data
Read in housing data from "Open Minneapolis" website using APIs
Store in dataframe

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
import time
from scipy.stats import linregress
import json
from pprint import pprint
from datetime import datetime



In [ ]:
#Note this does not return the full data set: 


#This seems to return the full dataset
#house_sales_geojson = 'https://opendata.arcgis.com/datasets/de791d30db6849989db8eeced93b78fb_0.geojson'

#prop_tax_geoserv = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Assessors_Parcel_Data_2020/FeatureServer/0/query?outFields=*&where=1%3D1'

#prop_tax_geojson = 'https://opendata.arcgis.com/datasets/bfd1c51a341f431c82ae5c56f852cbbb_0.geojson'

house_sales_geoserv = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Property_Sales_2015_to_2019/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'

msp_housing_data = requests.get(house_sales_geoserv).json()


In [ ]:
num_houses = len(msp_housing_data['features'])

sale_id =[]
sale_date = []
formatted_address = []
land_sale = []
community_cd = []
community_desc = []
nbhd_cd = []
nbhd_desc = []
ward = []
proptype_cd = []
proptype_desc = []
adj_sale_price = []
gross_sale_price = []
downpayment = []
x_coord = []
y_coord = []

for house in range(0, num_houses):
    sale_id.append(msp_housing_data['features'][house]['attributes']['SALE_ID'])
    sale_date.append(msp_housing_data['features'][house]['attributes']['SALE_DATE'])
    formatted_address.append(msp_housing_data['features'][house]['attributes']['FORMATED_ADDRESS'])
    land_sale.append(msp_housing_data['features'][house]['attributes']['LAND_SALE'])
    community_cd.append(msp_housing_data['features'][house]['attributes']['COMMUNITY_CD'])
    community_desc.append(msp_housing_data['features'][house]['attributes']['COMMUNITY_DESC'])
    nbhd_cd.append(msp_housing_data['features'][house]['attributes']['NBHD_CD'])
    nbhd_desc.append(msp_housing_data['features'][house]['attributes']['NBHD_DESC'])
    ward.append(msp_housing_data['features'][house]['attributes']['WARD'])
    proptype_cd.append(msp_housing_data['features'][house]['attributes']['PROPTYPE_CD'])
    proptype_desc.append(msp_housing_data['features'][house]['attributes']['PROPTYPE_DESC'])
    adj_sale_price.append(msp_housing_data['features'][house]['attributes']['ADJ_SALE_PRICE'])
    gross_sale_price.append(msp_housing_data['features'][house]['attributes']['GROSS_SALE_PRICE'])
    downpayment.append(msp_housing_data['features'][house]['attributes']['DOWNPAYMENT'])
    x_coord.append(msp_housing_data['features'][house]['attributes']['X'])
    y_coord.append(msp_housing_data['features'][house]['attributes']['Y'])



In [124]:
msp_home_sales_df = pd.DataFrame(zip(sale_id, sale_date, formatted_address, land_sale, community_cd, community_desc, nbhd_cd, nbhd_desc, ward, proptype_cd, proptype_desc, adj_sale_price, gross_sale_price, downpayment, x_coord, y_coord), columns = ['sale_id', 'sale_date', 'formatted_address', 'land_sale', 'community_cd', 'community_desc', 'nbhd_cd', 'nbhd_desc', 'ward', 'proptype_cd', 'proptype_desc', 'adj_sale_price', 'gross_sale_price', 'downpayment', 'x_coord', 'y_coord'])

msp_home_sales_df.set_index('sale_id', inplace=True)


In [125]:
msp_home_sales_df.head()

,sale_date,formatted_address,land_sale,community_cd,community_desc,nbhd_cd,nbhd_desc,ward,proptype_cd,proptype_desc,adj_sale_price,gross_sale_price,downpayment,x_coord,y_coord
sale_id,,,,,,,,,,,,,,,
304279,1420783200000,3542 Valley St,NO,2,NORTHEAST,11,COLUMBIA,1,R,RESIDENTIAL,145985,150500,7525,533346.190261,188201.516808
303578,1420783200000,301 Oak Grove St #506,NO,4,CENTRAL,31,LORING PARK,7,X,CONDOMINIUM,241000,241000,0,525967.100955,163990.980501
304190,1420783200000,3136 West Bde Maka Ska Blvd W #206,NO,6,CALHOUN-ISLE,49,WEST CALHOUN,13,X,CONDOMINIUM,20000,20000,0,516063.000000,156554.000000
303562,1420783200000,730 4TH St N #601,NO,4,CENTRAL,86,NORTHLOOP,3,X,CONDOMINIUM,320000,320000,0,526709.800000,171382.570000
304311,1420783200000,3439 Polk St,NO,2,NORTHEAST,12,WAITE PARK,1,R,RESIDENTIAL,153500,160000,18741,535920.652943,187323.980831
